In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
females=pd.read_csv('female_names.csv',na_values=['MARIA CONCEIÇAO',
                                                     'ESPERANÇA',
                                                     'FRANÇOISE',
                                                     'MARIA ESPERANÇA',
                                                     'DOLÇA',
                                                     'MERÇE',
                                                     'MARIE FRANÇOISE',
                                                     'MARIA GRAÇA',
                                                     'CONCEIÇAO',
                                                     'CONSTANÇA',
                                                     'FRANÇOISE MARIE',
                                                     'MARIA GRAÇAS',
                                                     "M'BARKA",
                                                     'MARIA MERÇE',
                                                     'MARIA DOLÇA',
                                                     'SANÇA',
                                                     'FATIMA CONCEIÇAO',
                                                             'nan']).dropna()

In [3]:
females.sort_values('frequency', ascending=False, inplace=True)

In [4]:
names=[str(x)+'.' for x in females.name]

In [5]:
distinct=[]
for name in names:
    for char in name:
        if char not in distinct:
            distinct.append(char)

In [6]:
len(distinct)

29

In [7]:
distinct=sorted(distinct)

In [8]:
char2ix={c:i for i,c in enumerate(distinct)}
ix2char={i:c for i,c in enumerate(distinct)}

In [9]:
X=[]
for name in names:
    #print(name)
    encoding=[char2ix[c] for c in name]
    number_name=''.join(str(encoding))
    #print(number_name)
    x=np.zeros((1,len(encoding)+1, 29),dtype='float32')
    for n, ix in enumerate(encoding):
        x[0,n+1,ix]=1
    #print(x)
    X.append(x)

In [10]:
X[0]

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.

In [11]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])

In [12]:
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Model

In [13]:
lstm=LSTM(64, return_sequences=True)
densor=Dense(29, activation='softmax')

In [14]:
class miRNN:
    def forward(self, inputs):
        x = lstm(inputs)
        x = densor(x)
        return x
    
    def give_model(self):
        inputs = Input(shape=(None,29))
        outputs = self.forward(inputs)
        
        model = Model(inputs=inputs, outputs=outputs)
        
        return model

In [15]:
model = miRNN().give_model()

In [16]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 29)]        0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 64)          24064     
_________________________________________________________________
dense (Dense)                (None, None, 29)          1885      
Total params: 25,949
Trainable params: 25,949
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
Y=[]
for i,x in enumerate(X):
    Y.append(x[0,1:,:].reshape(1,-1,29))
    X[i]=x[0,:-1,:].reshape(1,-1,29)

In [22]:
for epoch in range(5):
    print(f'Training on the {epoch+1} epoch')
    for i in range(len(X)):
        model.fit(X[i],Y[i],epochs=1,verbose=0)

Training on the 1 epoch
Training on the 2 epoch
Training on the 3 epoch
Training on the 4 epoch
Training on the 5 epoch


In [177]:
name_test='ANA MARIA ALEJANDRA'
name_test_list=[char2ix[c] for c in name_test]
name_test_final=np.zeros((1,len(name_test),29))
for n,char in enumerate(name_test_list):
    name_test_final[0,n,char]=1

prediction=model.predict(name_test_final)

([ix2char[int(n)] for n in  np.argmax(prediction,axis=2).reshape(-1)])

['N',
 'A',
 ' ',
 'M',
 'A',
 'R',
 'I',
 'A',
 ' ',
 'A',
 'L',
 'E',
 'J',
 'A',
 'N',
 'D',
 'R',
 'A',
 '.']

In [448]:
np.random.seed(k)
inicial=np.zeros((1,1,29))
name=inicial
nombre=''
for n in range(40):
    next_index=np.random.choice(range(29), p=model.predict(name)[:,-1,:].reshape(-1))
    next_char=ix2char[next_index]
    print(next_char)
    nombre+=next_char
    next_vector=np.zeros((1,1,29))
    next_vector[0,0,next_index]=1
    name=np.concatenate((name,next_vector),axis=1)
    if next_char == '.':
        break
print(nombre, k)
k+=1

S
A
N
D
R
A
 
V
A
L
E
N
T
I
N
A
.
SANDRA VALENTINA. 195


In [449]:
nombre in names

False

In [358]:
model.save('nombresGeneratorF')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: nombresGeneratorF/assets
